In [ ]:
# import necessary libraries
import requests
import sys
import numpy as np
import pandas as pd
import time
import random
import re
from bs4 import BeautifulSoup

In [ ]:
# scraping box office mojo
scraped = pd.DataFrame() # instantiate an empty dataframe
for i in range(1970,2021,1): # we want box office data for 1970 - 2020
    print(i)
    time.sleep(random.randrange(1,5,1))
    page = requests.get("https://www.boxofficemojo.com/year/world/{}/".format(i)) # go to the box office page for our year
    soup = BeautifulSoup(page.content, 'html.parser') # get page content
    for i in range(100): # we want the top 100 movies of each year
        try:
            href = soup.select("#table .a-link-normal:not(.a-nowrap)")[i]['href'] # get the url for the specific movie
            movie_title = soup.select("#table .a-link-normal:not(.a-nowrap)")[i].text
            time.sleep(random.randrange(1,5,1))
            page = requests.get("https://www.boxofficemojo.com{}".format(href)) # go to specific movie data url
            soup2 = BeautifulSoup(page.content, 'html.parser') # get page content
            href = soup2.select("table .a-link-normal")[0]['href'] # get link for domestic data because it has daily data
        except:
            print('Movie {} errored out in try block 1'.format(i + 1))
            continue

        try:
            time.sleep(random.randrange(1,5,1))
            page = requests.get("https://www.boxofficemojo.com{}".format(href)) # go to domestic data url
            soup3 = BeautifulSoup(page.content, 'html.parser') # get page content
            genres = soup3.select(".mojo-summary-values div")[6].text.replace('\n','') # get movie genres
            genres = re.sub(r'\s+',' ',genres) # clean genre text
            genres = re.sub('Genres', 'Genres ', genres) # clean genre text, ready for list
            genres = genres.split() # make into list
            genres = genres[1:] # get rid of the index that say genres
        except:
            print('Movie {} errored out in try block 2'.format(i + 1))
            continue       

        try:
            data = [] # instantiate empty data list
            # regex pattern to capture wanted information
            pattern = r'([a-zA-Z]{3}\s\d{1,2},\s[12][09]\d{2})(?:.*Day|.*Eve|Easter Sunday|Thanksgiving|Halloween)?([a-zA-Z]*)([\d]*)\$([\w,]*)(-|(?:[+-<][\d\.,]+))[%]?(-|(?:[+-<][\d\.,]+))[%]?([\d,]+)\$([\d,]+)\$((?:[\d]{0,3},)*[\d]{3})(\d{1,4})'
            for x in soup3.select("#table tr")[1:]: # loop through each daily box office row
                string = x.text # get only the text
                data.append(list(re.findall(pattern, string)[0])) # use regex to get the info we want
            # turn data into data frame and append to our dataframe
            df = pd.DataFrame(data, columns = ['date','dow','rank','daily_revenue','+/-YD','+/-LW','theaters','theater_avg','revenue_to_date','week'])
            df['movie_title'] = movie_title
            df['genre'] = [genres for x in range(df.shape[0])]
            scraped = pd.concat([scraped.copy(), df], ignore_index = True)
        except:
            print('Movie {} errored out in try block 3: data is weekly instead of daily'.format(i + 1))
            continue
        print(i+1)
        scraped.to_csv('scraped.csv') # save the info after every iteration